In [ ]:
import numpy as np
import os
import tensorflow as tf

from models import ConvNet, SoftBinaryDecisionTree
from models.utils import brand_new_tfsession, draw_tree
from tensorflow.keras.callbacks import EarlyStopping, Callback

sess = brand_new_tfsession()

In [ ]:
#上采样
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import *
# from keras.layers import *
import tensorflow as tf
import numpy as np
from sklearn import preprocessing
import xlwt

def normalization(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range

original_data = pd.read_csv('./dataset/SOFTLAB/ar3.csv')
# original_data = pd.read_csv('./dataset/bugzilla.csv')
original_data.isnull().values.any()  # Gives false ie:No null value in dataset
original_data = original_data.fillna(value=False)  #将缺失值填充为False
# original_Y = original_data['defects']  #Defective   class   isDefective  defects
original_Y = original_data['defects']
original_Y = pd.DataFrame(original_Y)
original_data = normalization(original_data)

#  将数据写入新文件  
#original_data.to_excel('C:/Users/lenovo/Desktop/excel/ar6.xls',sheet_name='ar6',index=False)
# original_X = pd.DataFrame(original_data.drop(['defects'], axis=1))  #Defective   class  isDefective  defects
original_X = pd.DataFrame(original_data.drop(['defects'], axis=1))
#print(original_X)
#分为训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(original_X, original_Y, test_size=.1, random_state=12)
# now we resample, and from that we take training and validation sets
sm = SMOTE(random_state=12, sampling_strategy=1.0) # 解决分类不平衡问题
x, y = sm.fit_resample(x_train, y_train)
y_train = pd.DataFrame(y, columns=['defects']) #Defective  class  isDefective  defects
x_train = pd.DataFrame(x, columns=original_X.columns)
#print(x_test) 

#对单个数据进行加量 
#x_test.loc[1,'total_loc'] =  0.264528801 
#x_test.loc[1,'comment_loc'] =  0.337140831 
#x_test.loc[1,'executable_loc'] =  0.203275228 
#x_test.loc[1,'unique_operands'] =  0.369565217 
#x_test.loc[1,'unique_operators'] =  0.479770399 
#x_test.loc[1,'total_operands'] =  0.282879173 
#x_test.loc[1,'total_operators'] =  0.250517298 
#x_test.loc[1,'halstead_vocabulary'] =  0.414459875 
#x_test.loc[1,'halstead_length'] =  0.262842023 
#x_test.loc[1,'halstead_volume'] =  0.244372278 
#x_test.loc[1,'halstead_level'] =  0.500382013 
#x_test.loc[1,'halstead_difficulty'] =  0.280770116 
#x_test.loc[1,'halstead_effort'] =  0.152186111 
#x_test.loc[1,'halstead_time'] =  0.152185296 
#x_test.loc[1,'branch_count'] =  0.172317183 
#x_test.loc[1,'condition_count'] =  0.171775777 
#x_test.loc[1,'cyclomatic_complexity'] =  0.179914274 
#x_test.loc[1,'cyclomatic_density'] =  0.646325185 
#x_test.loc[1,'decision_density'] =  0.290946038 
#x_test.loc[1,'design_density'] =  0.200977818 
#x_test.loc[1,'normalized_cyclomatic_complexity'] =  0.455357667 

#细分出验证集
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.1, random_state=12)

x_train = x_train.values
x_val = x_val.values
x_test = x_test.values
y_train = y_train.values
y_val = y_val.values
y_test = y_test.values

img_rows, img_cols = 13,2 #7 3   13,2
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)

In [ ]:
img_rows, img_cols, img_chans, n_classes = 13,2, 1, 2  #7,3   13,2
print(img_rows, img_cols, img_chans, n_classes)

# retrieve image and label shapes from training data

In [ ]:
# convert labels to 1-hot vectors
y_train = tf.keras.utils.to_categorical(y_train, n_classes)
y_val = tf.keras.utils.to_categorical(y_val, n_classes)
y_test = tf.keras.utils.to_categorical(y_test, n_classes)

print(y_train.shape, y_val.shape, y_test.shape)
#print(y_train)

In [5]:
# normalize inputs and cast to float
x_train = (x_train / np.max(x_train)).astype(np.float32)
x_val = (x_val / np.max(x_val)).astype(np.float32)
x_test = (x_test / np.max(x_test)).astype(np.float32)

In [ ]:
epochs = 40
batch_size = 2
# data_valid = (x_val, y_val)
nn = ConvNet(img_rows, img_cols, img_chans, n_classes)
nn.maybe_train(data_train=(x_train, y_train),
               data_valid=(x_val, y_val),
               batch_size=4, epochs=40)  #batch_size=4 2
# nn.maybe_train(x_train, y_train, validation_data=data_valid,batch_size=batch_size, epochs=epochs)
nn.evaluate(x_train, y_train)

In [ ]:
nn.evaluate(x_val, y_val)
nn.evaluate(x_test, y_test)

In [ ]:
y_train_soft = nn.predict(x_train)
# print(y_train_soft)
print(y_test.shape)

In [ ]:
from sklearn import metrics

y = nn.predict(x_test)

y = np.argmax(y,axis=1)
y_test = np.argmax(y_test,axis=1)

accuracy_score=metrics.accuracy_score(y_test,y)

print("accuracy_score",accuracy_score)

precision=metrics.precision_score(y_test,y,average='macro')

print("precision=",precision)#precision对比可以，超过图中方法

recall=metrics.recall_score(y_test,y,average='macro')
print("recall=",recall)#recall比其他两种方法都要低

fscore=metrics.f1_score(y_test,y,average='macro')
print("f-score=",fscore)

In [ ]:
x_train_flat = x_train.reshape((x_train.shape[0], -1))
x_val_flat = x_val.reshape((x_val.shape[0], -1))
x_test_flat = x_test.reshape((x_test.shape[0], -1))

x_train_flat.shape, x_val_flat.shape, x_test_flat.shape

In [11]:
n_features = img_rows * img_cols * img_chans
tree_depth = 4
penalty_strength = 1e+1 
penalty_decay = 0.25
ema_win_size = 1000
inv_temp = 0.01
learning_rate = 1e-06  
batch_size = 2 

In [ ]:
sess = brand_new_tfsession(sess)

tree1 = SoftBinaryDecisionTree(tree_depth, n_features, n_classes,
    penalty_strength=penalty_strength, penalty_decay=penalty_decay,
    inv_temp=inv_temp, ema_win_size=ema_win_size, learning_rate=learning_rate)
tree1.build_model()

In [ ]:
import glob

epochs = 40

es = EarlyStopping(monitor='val_acc', patience=20, verbose=1)

tree1.maybe_train(
    sess=sess, data_train=(x_train_flat, y_train), data_valid=(x_val_flat, y_val),
    batch_size=batch_size, epochs=epochs, callbacks=[es])

In [ ]:
tree1.evaluate(x=x_val_flat, y=y_val, batch_size=2)
tree1.evaluate(x=x_test_flat, y=y_test, batch_size=2)

In [ ]:
sess = brand_new_tfsession(sess)

tree1 = SoftBinaryDecisionTree(tree_depth, n_features, n_classes,
    penalty_strength=penalty_strength, penalty_decay=penalty_decay,
    inv_temp=inv_temp, ema_win_size=ema_win_size, learning_rate=learning_rate)
tree1.build_model()

In [ ]:
epochs = 40

es = EarlyStopping(monitor='val_acc', patience=20, verbose=1)

tree1.maybe_train(
    sess=sess, data_train=(x_train_flat, y_train_soft), data_valid=(x_val_flat, y_val),
    batch_size=batch_size, epochs=epochs, callbacks=[es], distill=True)


In [ ]:
tree1.evaluate(x=x_val_flat, y=y_val, batch_size=1)

tree1.evaluate(x=x_test_flat, y=y_test, batch_size=1)

In [18]:
#度量元灵敏度计算

#增量为方差
loss   = 12.62952525799091   #53.85
loss1  = 12.629525331350473
loss2  = 12.629525184631348
loss3  = 12.629525111271786
loss4  = 12.629525184631348
loss5  = 12.629525331350473
loss6  = 12.629525184631348
loss7  = 12.629525331350473
loss8  = 12.629525331350473
loss9  = 12.62952525799091
loss10 = 12.62952525799091
loss11 = 12.629525184631348
loss12 = 12.629525184631348
loss13 = 12.629525331350473
loss14 = 12.629525331350473
loss15 = 12.629525404710035
loss16 = 12.62952525799091
loss17 = 12.62952525799091
loss18 = 12.629525184631348
loss19 = 12.629525331350473    #46.15% 
loss20 = 12.629525184631348
loss21 = 12.62952525799091

Sensitivity1 = (loss1 - loss) / 0.21076536
Sensitivity2 = (loss2 - loss) / 0.203807498
Sensitivity3 = (loss3 - loss) / 0.190775228
Sensitivity4 = (loss4 - loss) / 0.217391304
Sensitivity5 = (loss5 - loss) / 0.244476282
Sensitivity6 = (loss6 - loss) / 0.205956096
Sensitivity7 = (loss7 - loss) / 0.200793541
Sensitivity8 = (loss8 - loss) / 0.228019197
Sensitivity9 = (loss9 - loss) / 0.202439338
Sensitivity10 = (loss10 - loss) / 0.199596158
Sensitivity11 = (loss11 - loss) / 0.252959332
Sensitivity12 = (loss12 - loss) / 0.197256351
Sensitivity13 = (loss13 - loss) / 0.146958898
Sensitivity14 = (loss14 - loss) / 0.146958831
Sensitivity15 = (loss15 - loss) / 0.172317183
Sensitivity16 = (loss16 - loss) / 0.171775777
Sensitivity17 = (loss17 - loss) / 0.179914274
Sensitivity18 = (loss18 - loss) / 0.22079327
Sensitivity19 = (loss19 - loss) / 0.290946038
Sensitivity20 = (loss20 - loss) / 0.200977818
Sensitivity21 = (loss21 - loss) / 0.221315114

print(Sensitivity1)
print(Sensitivity2)
print(Sensitivity3)
print(Sensitivity4)
print(Sensitivity5)
print(Sensitivity6)
print(Sensitivity7)
print(Sensitivity8)
print(Sensitivity9)
print(Sensitivity10)
print(Sensitivity11)
print(Sensitivity12)
print(Sensitivity13)
print(Sensitivity14)
print(Sensitivity15)
print(Sensitivity16)
print(Sensitivity17)
print(Sensitivity18)
print(Sensitivity19)
print(Sensitivity20)
print(Sensitivity21)


3.4806271655352e-07
-3.599453538290518e-07
-7.690680047896341e-07
-3.374539856507957e-07
3.0006822402911303e-07
-3.56190292034977e-07
3.653482248065968e-07
3.2172538418763314e-07
0.0
0.0
-2.9000535936196956e-07
-3.7189962000576484e-07
4.991842260342794e-07
4.991844536173576e-07
8.514480284743533e-07
0.0
0.0
-3.3225452017003854e-07
2.5214147702874236e-07
-3.6501322738325167e-07
0.0
